# **Setup**

In [1]:
import pandas as pd
# import re
# pip install nltk
# import nltk
# nltk.download()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from string import punctuation
punctuation = list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

Import data

In [2]:
df_combined = pd.read_excel("streaming_service_titles.xlsx", index_col=False);
df_prep = df_combined.copy();

In [3]:
df_prep[1:2]

,type,title,release_year,rating,genres,description,Streaming Service,Textual Info
1,TV Show,"""The Paramedic Angel""",2021,ALL,Drama,The tragedy of a loving family man and paramed...,Amazon Prime,drama tragedy loving family man paramedic hero...


# **Analysis**

In [4]:
indices = pd.Series(df_prep.index, index=df_prep["title"]).drop_duplicates();

## TFIDF

In [5]:
# tfidf = TfidfVectorizer(strip_accents="ascii", stop_words="english", min_df=0.0005, sublinear_tf=True);
# tfidf = TfidfVectorizer(max_df=.65, min_df=1, stop_words="english", use_idf=True, norm=None);
tfidf = TfidfVectorizer(stop_words="english", min_df=0.005, sublinear_tf=True);
tfidf_matrix = tfidf.fit_transform(df_prep["Textual Info"]);

In [6]:
cosine_simTFIDF = linear_kernel(tfidf_matrix, tfidf_matrix);

In [7]:
def get_recommendations(title, cosine_sim):
    idx = indices[title];
    sim_scores = list(enumerate(cosine_sim[idx]));
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True);
    sim_scores = sim_scores[1:11];
    movie_indices = [i[0] for i in sim_scores];
    dataframe1 = df_prep[["title","description","Streaming Service"]].iloc[movie_indices];
    # dataframe1["Similarity Score"] =  sim_scores;
    # print(type(sim_scores[3][1]))
    # print(sim_scores[3][1])
    similarityScores = [];
    for i in range(len(sim_scores)):
        similarityScores.append(sim_scores[i][1]);
    dataframe1["Similarity Score"] =  similarityScores;
    dataframe1 = dataframe1.reset_index().rename(columns={"index":"id"});
    return dataframe1;

In [8]:
get_recommendations("Ganglands", cosine_simTFIDF)

,id,title,description,Streaming Service,Similarity Score
0,12733,Narcos,The true story of Colombia's infamously violen...,Netflix,0.635364
1,3542,Cannabis,When a huge marijuana shipment falls prey to t...,Netflix,0.563305
2,2501,Better Than Us,A family on the brink of splitting up become t...,Netflix,0.555709
3,4207,Cocaine,Three films chronicle the cocaine trade's swee...,Netflix,0.554274
4,16017,She,An undercover assignment to expose a drug ring...,Netflix,0.548441
5,12734,Narcos: Mexico,Witness the birth of the Mexican drug war in t...,Netflix,0.541296
6,9936,Killer Ratings,Brazilian TV personality and politician Wallac...,Netflix,0.540868
7,12033,Mob Psycho 100,There's an organization gathering espers for a...,Netflix,0.531299
8,14657,Rake,While Cleaver Greene is a brilliant and driven...,Netflix,0.526270
9,5920,El final del paraíso,"In Colombia, the DEA's new director targets a ...",Netflix,0.523722


## N Grams
This did not actually run but due to filtering of the data via the Dash app components, we were able to implement NGrams text mining method

In [9]:
vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\b[a-zA-Z]{3,}\b', ngram_range=(1, 1));
matrix = vectorizer.fit_transform(df_prep["Textual Info"]);

In [10]:
cosine_simNGram = cosine_similarity(matrix, matrix);

MemoryError: Unable to allocate 3.85 GiB for an array with shape (22741, 22741) and data type float64

In [11]:
get_recommendations("Ganglands", cosine_simNGram)

NameError: name 'cosine_simNGram' is not defined